### Logistic Regression

#### Import Packages

In [52]:
%run ml_helpers.ipynb
%run ml.ipynb

         Unnamed: 0  Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  \
1123301     1123301  1731  2019      3   Third  0.00   0.0    47    31   
1143637     1143637   114  2019      4   First  0.00   0.0    78    45   
1210722     1210722  2221  2019      7   Third  0.40   0.0    84    72   
1287552     1287552   123  2019     10  Second  3.45   0.0    51    39   
1289224     1289224   632  2019     10   Third  0.00   0.0    52    41   
...             ...   ...   ...    ...     ...   ...   ...   ...   ...   
1330332     1330332   422  2019     12   Third  0.85   0.0    58    48   
1079227     1079227  1533  2019      1   First  0.21   3.3    32    30   
1168768     1168768  1925  2019      5   First  0.00   0.0    83    72   
1143310     1143310  1831  2019      4   Third  0.00   0.0    65    41   
1125608     1125608  2424  2019      3   First  0.00   0.0    46    27   

         count_l_stops  ...  count_restaurants  count_bars  count_daycares  \
1123301            0.0  ...      

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [17]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.decomposition import PCA
import datetime

#### Read and Process Data

In [53]:
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
data = data.drop(columns=["beat", "beat_num", "Crimes", "Arrest", "Domestic", "Serious"])
data = convert_to_categorical(data, ["district", "sector",
                                    "Month",
                                     "Watch", "Beat"])
data.drop(columns = ["Unnamed: 0"], inplace=True)


In [54]:
data.dtypes

district               category
sector                 category
Year                      int64
Month                  category
Watch                  category
Beat                   category
TMAX                    float64
TMIN                    float64
PRCP                    float64
SNOW                    float64
high_crime              float64
count_l_stops           float64
count_bus_stops         float64
count_metra_stops       float64
count_restaurants       float64
count_bars              float64
count_daycares          float64
count_entertainment     float64
count_businesses        float64
road_distance_ft        float64
TOTAL POPULATION        float64
dist_to_police          float64
dist_to_hospital        float64
high_crime_geog_lag     float64
dtype: object

#### Setup ML

In [55]:
models = {
    'LogisticRegression': LogisticRegression(), 
}
grid = {
    'LogisticRegression': [{'penalty': x, 'C': y, 'random_state': 0} 
                           for x in ('l2', 'none') \
                           for y in (0.001, 0.01, 0.1, 1, 10, 100, 1000)]
}

In [10]:
grid

{'LogisticRegression': [{'penalty': 'l2', 'C': 0.001, 'random_state': 0},
  {'penalty': 'l2', 'C': 0.01, 'random_state': 0},
  {'penalty': 'l2', 'C': 0.1, 'random_state': 0},
  {'penalty': 'l2', 'C': 1, 'random_state': 0},
  {'penalty': 'l2', 'C': 10, 'random_state': 0},
  {'penalty': 'l2', 'C': 100, 'random_state': 0},
  {'penalty': 'l2', 'C': 1000, 'random_state': 0},
  {'penalty': 'none', 'C': 0.001, 'random_state': 0},
  {'penalty': 'none', 'C': 0.01, 'random_state': 0},
  {'penalty': 'none', 'C': 0.1, 'random_state': 0},
  {'penalty': 'none', 'C': 1, 'random_state': 0},
  {'penalty': 'none', 'C': 10, 'random_state': 0},
  {'penalty': 'none', 'C': 100, 'random_state': 0},
  {'penalty': 'none', 'C': 1000, 'random_state': 0}]}

In [56]:
data_list = prep_data(data, "high_crime",
                                        2, "Year", ["district", "sector", 
                                                    "Month", 
                                                    "Beat", "Watch"])


      district sector  Year Month   Watch  Beat       TMAX       TMIN  PRCP  \
29557       17      1  2018     1   First  1713  33.201426  18.884223  1.12   
29558       17      1  2018     1  Second  1713  33.201426  18.884223  1.12   
29559       17      1  2018     1   Third  1713  33.201426  18.884223  1.12   
29560       17      1  2018     2   Third  1713  39.791635  22.439919  5.47   
29561       17      1  2018     2   First  1713  39.791635  22.439919  5.47   
...        ...    ...   ...   ...     ...   ...        ...        ...   ...   
49256        3      1  2019    11  Second   312  43.761013  29.648049  1.70   
49257        3      1  2019    11   Third   312  43.761013  29.648049  1.70   
49258        3      1  2019    12  Second   312  42.647269  27.854899  1.29   
49259        3      1  2019    12   First   312  42.647269  27.854899  1.29   
49260        3      1  2019    12   Third   312  42.647269  27.854899  1.29   

       SNOW  ... count_restaurants  count_bars  cou

Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [57]:
data_list[0][1]

,Year,TMAX,TMIN,PRCP,SNOW,high_crime,count_l_stops,count_bus_stops,count_metra_stops,count_restaurants,...,Beat_2524,Beat_2525,Beat_2531,Beat_2532,Beat_2533,Beat_2534,Beat_2535,Watch_First,Watch_Second,Watch_Third
39407,2019,-1.638995,-1.575662,-0.754929,3.894522,0.0,3.213582,-0.590245,-0.438065,0.362205,...,0,0,0,0,0,0,0,1,0,0
39408,2019,-1.638995,-1.575662,-0.754929,3.894522,0.0,3.213582,-0.590245,-0.438065,0.362205,...,0,0,0,0,0,0,0,0,1,0
39409,2019,-1.638995,-1.575662,-0.754929,3.894522,0.0,3.213582,-0.590245,-0.438065,0.362205,...,0,0,0,0,0,0,0,0,0,1
39410,2019,-1.449889,-1.501326,-0.650552,0.957963,0.0,3.213582,-0.590245,-0.438065,0.362205,...,0,0,0,0,0,0,0,0,1,0
39411,2019,-1.449889,-1.501326,-0.650552,0.957963,0.0,3.213582,-0.590245,-0.438065,0.362205,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49256,2019,-0.939117,-0.914468,-0.850956,0.087121,0.0,1.951714,-0.251472,-0.438065,-0.704410,...,0,0,0,0,0,0,0,0,1,0
49257,2019,-0.939117,-0.914468,-0.850956,0.087121,0.0,1.951714,-0.251472,-0.438065,-0.704410,...,0,0,0,0,0,0,0,0,0,1
49258,2019,-0.999382,-1.020879,-1.022134,-0.135653,1.0,1.951714,-0.251472,-0.438065,-0.704410,...,0,0,0,0,0,0,0,0,1,0
49259,2019,-0.999382,-1.020879,-1.022134,-0.135653,0.0,1.951714,-0.251472,-0.438065,-0.704410,...,0,0,0,0,0,0,0,1,0,0


In [58]:
data_list_pca = prep_data(data, "high_crime",
                                        2, "Year", 
                          ["district", "sector", 
                                                    "Month", 
                                                    "Beat", "Watch"], pca="yes!", 
                                                  columns_to_pca = ["TOTAL POPULATION", "dist_to_police",
                                                                   "dist_to_hospital", 
                                                  "count_l_stops", "count_bus_stops", 
                                               "count_metra_stops", "count_restaurants", "count_bars", 
                                               "count_daycares", "count_entertainment", "count_businesses", 
                                               "road_distance_ft"])

      district sector  Year Month   Watch  Beat       TMAX       TMIN  PRCP  \
29557       17      1  2018     1   First  1713  33.201426  18.884223  1.12   
29558       17      1  2018     1  Second  1713  33.201426  18.884223  1.12   
29559       17      1  2018     1   Third  1713  33.201426  18.884223  1.12   
29560       17      1  2018     2   Third  1713  39.791635  22.439919  5.47   
29561       17      1  2018     2   First  1713  39.791635  22.439919  5.47   
...        ...    ...   ...   ...     ...   ...        ...        ...   ...   
49256        3      1  2019    11  Second   312  43.761013  29.648049  1.70   
49257        3      1  2019    11   Third   312  43.761013  29.648049  1.70   
49258        3      1  2019    12  Second   312  42.647269  27.854899  1.29   
49259        3      1  2019    12   First   312  42.647269  27.854899  1.29   
49260        3      1  2019    12   Third   312  42.647269  27.854899  1.29   

       SNOW  ... count_restaurants  count_bars  cou

Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
0    0.852368
1    0.852368
2    0.852368
3    0.852368
4    0.852368
Name: principal component 1, dtype: float64
(19695, 2) (19695, 322)
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
0    0.85269
1    0.85269
2    0.85269
3    0.85269
4    0.85269
Name: principal component 1, dtype: float64
(19696, 2) (19696, 322)
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
0    0.856576
1    0.856576
2    0.856576
3    0.856576

In [59]:
logistic_results_pca = average_grid_searches(LogisticRegression(), "LogisticRegression", 
                          grid, 
                          data_list_pca, "high_crime", 2020)


test year is: 2020
[2016, 2017, 2018, 2019]


In [ ]:
logistic_results = average_grid_searches(LogisticRegression(), "LogisticRegression", 
                          grid, 
                          data_list, "high_crime", 2020)

In [12]:
logistic_results.to_csv("../final_data/log_reg_results.csv")

In [60]:
logistic_results_pca

,parameters_2019,precision_2019,parameters_2018,precision_2018,parameters_2017,precision_2017,mean
0,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}",0.763417,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}",0.786184,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}",0.732948,0.760850
0,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.749498,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.725907,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.746371,0.740592
0,"{'penalty': 'l2', 'C': 0.1, 'random_state': 0}",0.716374,"{'penalty': 'l2', 'C': 0.1, 'random_state': 0}",0.707344,"{'penalty': 'l2', 'C': 0.1, 'random_state': 0}",0.723997,0.715905
0,"{'penalty': 'none', 'C': 0.001, 'random_state'...",0.727001,"{'penalty': 'none', 'C': 0.001, 'random_state'...",0.696150,"{'penalty': 'none', 'C': 0.001, 'random_state'...",0.709322,0.710824
0,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.727001,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.696150,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.709322,0.710824
0,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.727001,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.696150,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.709322,0.710824
0,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.727001,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.696150,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.709322,0.710824
0,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.727001,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.696150,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.709322,0.710824
0,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.727001,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.696150,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.709322,0.710824
0,"{'penalty': 'none', 'C': 1000, 'random_state': 0}",0.727001,"{'penalty': 'none', 'C': 1000, 'random_state': 0}",0.696150,"{'penalty': 'none', 'C': 1000, 'random_state': 0}",0.709322,0.710824


In [44]:
logistic_results

,parameters_2020,precision_2020,parameters_2019,precision_2019,parameters_2018,precision_2018,parameters_2017,precision_2017,mean
0,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.674018,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.743653,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.722716,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.745098,0.721371
0,"{'penalty': 'l2', 'C': 0.1, 'random_state': 0}",0.654747,"{'penalty': 'l2', 'C': 0.1, 'random_state': 0}",0.723393,"{'penalty': 'l2', 'C': 0.1, 'random_state': 0}",0.705056,"{'penalty': 'l2', 'C': 0.1, 'random_state': 0}",0.732480,0.703919
0,"{'penalty': 'l2', 'C': 10, 'random_state': 0}",0.641465,"{'penalty': 'l2', 'C': 10, 'random_state': 0}",0.737089,"{'penalty': 'l2', 'C': 10, 'random_state': 0}",0.705755,"{'penalty': 'l2', 'C': 10, 'random_state': 0}",0.730567,0.703719
0,"{'penalty': 'l2', 'C': 100, 'random_state': 0}",0.651051,"{'penalty': 'l2', 'C': 100, 'random_state': 0}",0.714737,"{'penalty': 'l2', 'C': 100, 'random_state': 0}",0.705338,"{'penalty': 'l2', 'C': 100, 'random_state': 0}",0.725548,0.699168
0,"{'penalty': 'l2', 'C': 1, 'random_state': 0}",0.647702,"{'penalty': 'l2', 'C': 1, 'random_state': 0}",0.713605,"{'penalty': 'l2', 'C': 1, 'random_state': 0}",0.701998,"{'penalty': 'l2', 'C': 1, 'random_state': 0}",0.730083,0.698347
0,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.648004,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.714210,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.697990,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.726841,0.696761
0,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.648004,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.714210,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.697990,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.726841,0.696761
0,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.648004,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.714210,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.697990,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.726841,0.696761
0,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.648004,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.714210,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.697990,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.726841,0.696761
0,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.648004,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.714210,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.697990,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.726841,0.696761


Hello
